In [ ]:
# 구글 드라이브와 내 서버 연결 및 작업디렉토리 변경
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/KDT/세미프로젝트'

Mounted at /content/drive
/content/drive/MyDrive/KDT/세미프로젝트


## 파일 업로드

In [ ]:
### 필요 라이브러리 임포트

import pandas as pd
from collections import Counter
import re

In [ ]:
### 데이터 로드

df = pd.read_csv('kr3_raw.tsv', sep = '\t')
df.head()

,Region,Rating,Category,Review
0,인덕원,1,삼겹살 고기집,숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1,인덕원,1,삼겹살 고기집,고기가 정말 맛있었어요! 육즙이 가득있어서 너무 좋았아요\n일하시는분들 너무 친절하...
2,인덕원,1,순대국 찹쌀순대,"잡내없고 깔끔, 담백한 맛의 순대국이 순대국을 안 좋아하는 사람들에게도 술술 넘어갈..."
3,인덕원,1,순대국 찹쌀순대,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다~~ 맛도 좋습니다\n다만 양념...
4,인덕원,1,순대국 찹쌀순대,순대국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요.\n\n그러나 ...


In [ ]:
### 데이터 전처리
# 누락 데이터 확인
df.isnull().sum()

# 결측치 제거 및 제거 확인
df = df.dropna()
df.isnull().sum()

Region      0
Rating      0
Category    0
Review      0
dtype: int64

In [ ]:
### Rating 컬럼에서 Ambiguous(모호함)을 나타낸 값 '2'를 가진 행 제거

df = df[df['Rating'] != 2]
df = df.reset_index()
df.tail()

,index,Region,Rating,Category,Review
459522,642404,평택,0,일식/중식/세계음식,"7/31 배달 시켜 먹었구요, 거리상 1.8km 입니다. 배민에서 시켰고 정확히 5..."
459523,642405,평택,1,한식,송탄 미군부대 근처에 위치한 곳 원래 로컬맛집으로 되게 유명했는데 삼대천왕에 나온 ...
459524,642406,평택,1,한식,집에서 40키로 정도 떨어져 있는 곳인데도 몇달에 한번은 이거 먹으러 일부러 갑니다...
459525,642409,평택,0,한식,원래 글 안쓰는데 이거는 정말 다른분들 위해서 써야할것같네요 방금 포장주문 해서 왔...
459526,642410,평택,1,한식,"우리팀 단골집, 술먹고 다음 날 가면 푸짐하게 배불리 해장 할 수 있는곳, 주말도 ..."


In [ ]:
### 지정한 카테고리 데이터들을 변별
df_category = df.loc[(df['Category'].isin(['한식', '기타 한식', '삼겹살','고기/구이류', '고기 요리', '탕/찌개/전골','한정식/백반/전통 한식','갈비','한정식', '','냉면'])), ['Rating','Review']].reset_index(drop=True)
df_category

,Rating,Review
0,1,개인적으로 제일 좋아하는 냉면집\n진짜 집 근처였으면 일주일에 한번씩갈것같아요
1,1,방화동 구석의 정말 허름한 노포. 아저씨의 응대가 유쾌하고 살갑다. 손으로 쓴 메뉴...
2,1,아주머니 혼자하시는 식당이고 푸짐해요\n국물이 사골이라 맛있어요 순대는 쏘쏘해요
3,1,장어가 담백하면서 고소하고 맛있습니다. 비린맛이 적어 물리지 않고 맛있게 먹을 수 ...
4,1,"오픈 이벤트로 막국수 할인 중이며, 맛있고 푸짐하여 가족 외식으로 괜찮은 곳"
...,...,...
61448,1,리뷰에직원이 친절하지않다더니 정말이네요 선불이라 만원짜리내고 기다리고있는데 거지동냥...
61449,1,송탄 미군부대 근처에 위치한 곳 원래 로컬맛집으로 되게 유명했는데 삼대천왕에 나온 ...
61450,1,집에서 40키로 정도 떨어져 있는 곳인데도 몇달에 한번은 이거 먹으러 일부러 갑니다...
61451,0,원래 글 안쓰는데 이거는 정말 다른분들 위해서 써야할것같네요 방금 포장주문 해서 왔...


In [ ]:
# 필요한 라이브러리
import re

def preprocess(data):
    my_split = []

    for pre in data.iloc:
      data = (pre['Review'])
      # 특수 문자 제거 후 나머지 문자열은 공백 기준으로 분할
      split_data = re.sub('[#"+*!.,]', '', data).split(' ')

      for i in split_data:
        if len(i) >= 2:
          my_split.append(i)

    return my_split

In [ ]:
my_category= preprocess(df_category)
print(Counter(my_category[0:1000]))

Counter({'정말': 6, '맛도': 6, '먹을': 5, '좋다': 5, '있어서': 5, '있는': 4, '좋은': 4, '맛있고': 4, '조금': 4, '맛이': 4, '맛있음': 4, '맛있게': 3, '맛있어요': 3, '있어요': 3, '갔는데': 3, '냉면은': 3, '하지만': 3, '먹으면': 3, '있음': 3, '있고': 3, '많이': 3, '깔끔하고': 3, '먹고': 3, '맛있는': 3, '그냥': 3, '양이': 3, '맛있다': 3, '김치찌개': 3, '다른': 3, '괜찮은': 2, '가면': 2, '하는': 2, '궁금해': 2, '집이': 2, '그런지': 2, '따라': 2, '아닐까': 2, '생각했는데': 2, '아닌': 2, '주문했는데': 2, '생각이': 2, '들었다': 2, '답게': 2, '좀더': 2, '냉면': 2, '고기': 2, '맛의': 2, '특색이': 2, '만두를': 2, '분위기가': 2, '더욱': 2, '분위기': 2, '싶으면': 2, '해서': 2, '가서': 2, '근데': 2, '빨리': 2, '위한': 2, '가게': 2, '여기': 2, '모든': 2, '반찬': 2, '없는': 2, '가격은': 2, '있습니다': 2, '진짜': 2, '너무': 2, '많고': 2, '반찬도': 2, '좋아요': 2, '하기': 2, '백반이': 2, '맛에': 2, '추천': 2, '가성비': 2, '맛을': 2, '좋음': 2, '좋고': 2, '나와서': 2, '의왕역': 2, '근처': 2, '맛집': 2, '삼겹배달': 2, '개인적으로': 1, '제일': 1, '좋아하는': 1, '냉면집\n진짜': 1, '근처였으면': 1, '일주일에': 1, '한번씩갈것같아요': 1, '방화동': 1, '구석의': 1, '허름한': 1, '노포': 1, '아저씨의': 1, '응대가': 1, '유쾌하고': 1, '살갑다': 1, '손으로': 1, '메뉴판': 1, '본게': 1, '대체'

## 특수 문자 제거

In [ ]:
# 각 전처리 후, 200글자 이하의 리뷰 갯수 비율 확인하는 함수
def del_percent():
    under_200 = df_category.loc[df_category['length'] <= 200,'Review'].count()
    all_count = df_category['Review'].count()
    print('200글자 이하 리뷰갯수:', under_200)
    print(under_200 / all_count * 100)

In [ ]:
# 리뷰의 length 측정해서 컬럼으로 넣어줌
df_category['length'] = df_category['Review'].apply(lambda x: len(str(x)))
df_category['Review']=df_category['Review'].astype('str')

## \n (개행문자) 제거

In [ ]:
#1.'\n' -> ''
df_category['Review'] = df_category['Review'].apply(lambda x: x.replace('\n',''))
df_category['length'] = df_category['Review'].apply(lambda x: len(x))
del_percent()

200글자 이하 리뷰갯수: 52491
85.41649716043155


## 텍스트에 포함되어 있는 특수 문자 제거

In [ ]:
import re
# 2. ~, !, ., >
def cleanText(readData):


    text = re.sub('[-=+,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤]', '', readData)
    return text

df_category['Review'] = df_category['Review'].apply(cleanText)
df_category['length'] = df_category['Review'].apply(lambda x: len(x))
del_percent()

200글자 이하 리뷰갯수: 54305
88.36834654125917


## 의미없는 자음, 모음만 쓴 것 처리

In [ ]:
def cleanText2(readData):

    text = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', readData)
    return text
df_category['Review'] = df_category['Review'].apply(cleanText2)
df_category['length'] = df_category['Review'].apply(lambda x: len(x))
del_percent()

200글자 이하 리뷰갯수: 54380
88.49039103054366


## 이모티콘 제거

In [ ]:
def cleanText3(readData):

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', readData)
    return text

df_category['Review'] = df_category['Review'].apply(cleanText3)
df_category['length'] = df_category['Review'].apply(lambda x: len(x))
del_percent()

200글자 이하 리뷰갯수: 54381
88.4920182904008


## 글자수 cut

In [ ]:
df_category[df_category['length'] > 200].shape

(7072, 3)

In [ ]:
df_category = df_category[df_category['length'] <= 200]
df_category.shape

(54381, 3)

### 맞춤법 검사기

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-3n1mv5pf
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-3n1mv5pf
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4810 sha256=3883b23b11ccb3faff4a8d62c38fe55c163bdfe2efd4fcf4ec6c1d4d45242b29
  Stored in directory: /tmp/pip-ephem-wheel-cache-a4iq2hzs/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [ ]:
#맞춤법검사기
from hanspell import spell_checker

In [ ]:
def spell_check(review):
    result = spell_checker.check(review)
    return result.checked

In [ ]:
df_category['Review']=df_category['Review'].astype('str')

<ipython-input-21-79524506e62b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category['Review']=df_category['Review'].astype('str')


In [ ]:
df_category['review_spell_check'] = df_category['Review'].apply(spell_check)

<ipython-input-22-a198af65e64f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category['review_spell_check'] = df_category['Review'].apply(spell_check)


In [ ]:
df.head()

,index,Region,Rating,Category,Review
0,0,인덕원,1,삼겹살 고기집,숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1,1,인덕원,1,삼겹살 고기집,고기가 정말 맛있었어요! 육즙이 가득있어서 너무 좋았아요\n일하시는분들 너무 친절하...
2,2,인덕원,1,순대국 찹쌀순대,"잡내없고 깔끔, 담백한 맛의 순대국이 순대국을 안 좋아하는 사람들에게도 술술 넘어갈..."
3,3,인덕원,1,순대국 찹쌀순대,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다~~ 맛도 좋습니다\n다만 양념...
4,4,인덕원,1,순대국 찹쌀순대,순대국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요.\n\n그러나 ...


## 데이터 파일 내보내기

In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
df_category.to_excel('data_rescue.xlsx')
df_category.to_csv("data_rescue.csv")

In [ ]:
df_load = pd.read_csv('data_rescue.csv')
df_load

,Unnamed: 0,Rating,Review,length,review_spell_check
0,0,1,개인적으로 제일 좋아하는 냉면집진짜 집 근처였으면 일주일에 한번씩갈것같아요,41,개인적으로 제일 좋아하는 냉면집 진짜 집 근처였으면 일주일에 한 번씩 갈 것 같아요
1,1,1,방화동 구석의 정말 허름한 노포 아저씨의 응대가 유쾌하고 살갑다 손으로 쓴 메뉴판 ...,79,방화동 구석의 정말 허름한 노포 아저씨의 응대가 유쾌하고 살갑다 손으로 쓴 메뉴판 ...
2,2,1,아주머니 혼자하시는 식당이고 푸짐해요국물이 사골이라 맛있어요 순대는 쏘쏘해요,42,아주머니 혼자 하시는 식당이고 푸짐해요 국물이 사골이라 맛있어요 순대는 쏘쏘해요
3,3,1,장어가 담백하면서 고소하고 맛있습니다 비린맛이 적어 물리지 않고 맛있게 먹을 수 있어요,48,장어가 담백하면서 고소하고 맛있습니다 비린 맛이 적어 물리지 않고 맛있게 먹을 수 있어요
4,4,1,오픈 이벤트로 막국수 할인 중이며 맛있고 푸짐하여 가족 외식으로 괜찮은 곳,41,오픈 이벤트로 막국수 할인 중이며 맛있고 푸짐하여 가족 외식으로 괜찮은 곳
...,...,...,...,...,...
54376,61444,1,자연밥상세트중에 마누라밥상 먹었어요 이거 많이 드시는거같던데 바싹소불고기랑 고등어구...,115,자연밥상 세트 중에 마누라 밥상 먹었어요 이거 많이 드시는 거 같던데 바싹 소 불고...
54377,61445,1,한상차림으로 똭 나오고 거기에 다른 필요한 쌈류나 반찬들 눈치안보고 편하게 셀바에서...,123,한상차림으로 똭 나오고 거기에 다른 필요한 쌈류나 반찬들 눈치 안 보고 편하게 셀바...
54378,61446,1,여기 정말 진짜 맛있음 마지막에 넣는 마늘이 더해져 느끼하지 않은 국물 햄 종류도 ...,118,여기 정말 진짜 맛있음 마지막에 넣는 마늘이 더해져 느끼하지 않은 국물 햄 종류도 ...
54379,61447,1,가격대비 괜찮긴한데 가격을 자꾸 올리고 샐러드바 메뉴는 줄어드는 경향이 있어서 살짝...,76,가격 대비 괜찮긴 한데 가격을 자꾸 올리고 샐러드바 메뉴는 줄어드는 경향이 있어서 ...
